In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../RD-systems-and-test-benches/utils")
import model_fsolve as modf

import scipy.optimize as sco
import scipy.interpolate as sci

import simu as sim

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl


In [ ]:
path = r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs"
file_name = "\V4.5_MPEx4.xlsx"

In [ ]:
hw, par, cond = fe.initialize(path, file_name)


In [ ]:
list_Dv = [0.027e-3, 0.055e-3, 0.110e-3, 0.165e-3, 0.220e-3]
list_Vdot = np.array(list_Dv)*3600000
list_PL_CFD = [1067, 1279, 3861, 5072, 9573]
list_PL_model ,list_tabl = modf.PL_fsolve_range(par, cond, list_Dv, 1.1/4)
plt.plot(list_Vdot, np.array(list_PL_CFD)/1000, label = 'CFD')
plt.plot(list_Vdot, np.array(list_PL_model)/1000, label = 'model')
plt.xlabel('Vdot (L/h)')
plt.ylabel('PL (kPa)')
         
plt.xlim(0, 850)
plt.ylim(0, 11)

plt.legend()
plt.show()

In [ ]:
cond['Dv'] = 0.055e-3 
cond['Vdot'] = cond['Dv']*3600000 
tabl, res, df_PL, t, f = modf.PL_fsolve(par, cond)

In [ ]:
df = pd.read_csv(r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\CFD\V4.5_MPEx4\processed\0.055_exhaust_pressure.csv")
last = df["Position"].max() - 0.07
df["riser"] = [(df.loc[i]["Position"])*len(tabl)/last for i in range(len(df))]
df[df["Position"]<last].plot(x="riser", y="Pressure")
tabl["Pout"].plot(x=tabl.index)
point2 = df[df["Position"]<last].iloc[-1]["Pressure"]
df2=df[df["Position"]<last]

In [ ]:
df = pd.read_csv(r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\CFD\V4.5_MPEx4\processed\0.055_intake_pressure.csv")
last = df["Position"].max()
df["riser"] = [(df.loc[i]["Position"] - 0.074)*len(tabl)/(last- 0.074) for i in range(len(df))]
df[df["Position"]>0.074].plot(x="riser", y="Pressure")
tabl["Pin"].plot(x=tabl.index)
point1 = df[df["Position"]>0.074].iloc[0]["Pressure"]
df1=df[df["Position"]>0.074]

In [ ]:
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)
df1-df2

In [ ]:
D = '0.055'
df1 = pd.read_csv("G:/Drive partagés/BU04-Innovation/PVT-PL-model/CFD/V4.5_MPEx4/processed/" +D+ "_risers_entry_pressure_per_riser.csv")
df2 = pd.read_csv("G:/Drive partagés/BU04-Innovation/PVT-PL-model/CFD\V4.5_MPEx4/processed/"+D+"_risers_entry+0.050_pressure_per_riser.csv")
df3 = pd.read_csv("G:/Drive partagés/BU04-Innovation/PVT-PL-model/CFD/V4.5_MPEx4/processed/"+D+"_risers_after_first_bend_pressure_per_riser.csv")
df4 = pd.read_csv("G:/Drive partagés/BU04-Innovation/PVT-PL-model/CFD/V4.5_MPEx4/processed/"+D+"_risers_before_bj_pressure_per_riser.csv")   
df5 = pd.read_csv("G:/Drive partagés/BU04-Innovation/PVT-PL-model/CFD/V4.5_MPEx4/processed/"+D+"_risers_before_exhaust_pressure_per_riser.csv")

(df1["Pressure"]-df5["Pressure"]).plot(x="Group")
df_PL["RPL riser"].plot()

In [ ]:
df3

In [ ]:
PL = point1 - point2
PL

In [ ]:
((df1-df2)['Pressure']* (67/5)/(df3-df4)['Pressure']).mean()

In [ ]:
entry_5cm = np.array([0.7396875 - 0.7404128, 0.003751432 - 0.004154148, -0.3049988 + 0.5200261])
entry = np.array([0.7785593-0.7787191, 0.03597014-0.03598411, -0.5199889+0.3049711])
entry_5cm/entry

In [ ]:
hx, par, cond = fe.initialize(r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs",r"\V4.5_1MPE.xlsx")
list_QF = np.array([50,200,350,500,650,800])/3600000
list_QF_out = np.array([50,200,350,500])/3600000
list_alpha = [0.01, 0.05, 0.1, 0.4, 0.7, 1]
df = fe.testing_series(r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Outputs\Tests-model-simplification\V4.5_1MPE_testings.xlsx", list_QF, list_QF_out, list_alpha, par, cond)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

df_testings = pd.read_excel("G:\Drive partagés\BU04-Innovation\PVT-PL-model\Outputs\Tests-model-simplification\V4.5_1MPE_testings.xlsx")
X = df_testings[['QF', 'QF_out', 'alpha']].to_numpy()
yin = df_testings['DPin'].to_numpy()
yout = df_testings['DPout'].to_numpy()
yx = df_testings['DPx'].to_numpy()

model_in = RandomForestRegressor() 
model_in.fit(X, yin)

model_out = RandomForestRegressor() 
model_out.fit(X, yout)

model_x = RandomForestRegressor() 
model_x.fit(X, yx)

df_testings['DPin_test'] = model_in.predict(X)
df_testings['DPout_test'] = model_out.predict(X)
df_testings['DPx_test'] = model_x.predict(X)

In [ ]:
dplt.K_abaque(df_testings, 'DPout')

In [ ]:
hx, par, cond = fe.initialize(path, file_name)
list_Vdot = np.array([150, 300, 450, 600])
list_Dv = list_Vdot/3600000
for coef in [1, 1.1, 1.2, 1.4] :
    list_PL, list_tabl = modf.PL_fsolve_range(par, cond, list_Dv, 0.25*coef)
    plt.plot(list_Vdot, list_PL/1000, label = f'f *= {coef}')
plt.legend()
plt.show()
    


In [ ]:
df2 = pd.DataFrame(modf.calc(df['qx'], par, cond)).transpose()
df2.columns = ["Qin", "Qout", "uin", "ux", "uout", "Rein", "Rex", "Reout", "fin", "fx", "fout", "Kx_in", "Ky_in", "Kx_out", "Ky_out", "K_se", "Lex"]
df = df.join(df2)
df = df.join(df_PL)


In [ ]:
path = 'G:\Drive partagés\BU04-Innovation\PVT-PL-model\Outputs\Simulation_outputs'
path_inputs = 'G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs'
name = '\V4.41.xlsx'
list_Vdot = np.array([150, 300, 450, 600])
hx, par, cond = fe.initialize(path_inputs, name)

writer = pd.ExcelWriter(path+name, engine = 'openpyxl')
for Vdot in list_Vdot :
    cond['Vdot'] = Vdot
    cond['Dv'] = Vdot/3600000
    df, PL, df_PL, testings = modf.PL_fsolve(par, cond)
    df2 = pd.DataFrame(modf.calc(df['qx'], par, cond)).transpose()
    df2.columns = ["Qin", "Qout", "uin", "ux", "uout", "Rein", "Rex", "Reout", "fin", "fx", "fout", "Kx_in", "Ky_in", "Kx_out", "Ky_out", "K_se", "Lex"]
    df = df.join(df2)
    df = df.join(df_PL)
    df.to_excel(writer, sheet_name= f"Vdot={Vdot}", index=False)

writer.close()


In [ ]:
dplt.solve_plot(path, file_name)

In [ ]:
df_K = pd.read_excel("G:\Drive partagés\BU04-Innovation\PVT-PL-model\Outputs\Tests-model-simplification\V4.5_1MPE_testings.xlsx")

In [ ]:
df_A = df_K[['QF', 'QF_out','alpha']]
df_A['constant']=1
df_A['sqrt'] = np.sqrt(df_A['QF'])
df_A['QF*alpha'] = df_A['alpha']*df_A['QF']
df_A['sqrt_alpha'] = df_A['alpha']*np.sqrt(df_A['QF'])
df_A['QF_out*alpha'] = df_A['alpha']*df_A['QF_out']
df_A['alpha2'] = df_A['alpha']**2
df_A['QF*alpha2'] = df_A['alpha2']*df_A['QF']
df_A['sqrt_alpha2'] = df_A['alpha2']*np.sqrt(df_A['QF'])
df_A['QF_out*alpha2'] = df_A['alpha2']*df_A['QF_out']
df_A = df_A[['constant', 'QF', 'sqrt', 'QF_out', 'alpha', 'QF*alpha', 'QF_out*alpha', 'sqrt_alpha', 'alpha2', 'QF*alpha2','QF_out*alpha2', 'sqrt_alpha2']]

In [ ]:
A = df_A.to_numpy()
B = df_K[['Kyin']].to_numpy()
X = np.linalg.lstsq(A,B)[0]

err = (A.dot(X)-B)*100/B
np.abs(err.round()).mean()

In [ ]:
X.transpose()*(df_A.mean().to_numpy())

In [ ]:
df = pd.DataFrame(X.transpose())
df.to_excel("G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs\V4.5_1MPE_Kyin.xlsx", index=False)

In [ ]:
df_K['Kyin_test'] = A.dot(X)

In [ ]:
dplt.K_abaque(df_K,'Kyin')

In [ ]:
TUV = pd.read_excel('G:\Drive partagés\BU04-Innovation\Hydraulique\PL_database\\230727_TUV_V4.41_x1_Z_water.xlsx')
path = r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs"
file_name = r"\V4.41.xlsx"
hx, par, cond = fe.initialize(path, file_name)

In [ ]:
list_Vdot = np.array(list(TUV['Vdot']))[::2]
list_Dv = list_Vdot/3600000
f = 20
list_PL,list_tabl = modf.PL_fsolve_range(par, cond, list_Dv, f)
plt.plot(list_Vdot, list_PL/1000, label = f'fapp = {f}')
plt.plot(list(TUV['Vdot']), list(TUV['PL']), label = 'TÜV')
plt.legend()
plt.show()



In [ ]:
# read excel file
excel_file = pd.ExcelFile(r'G:\Drive partagés\BU04-Innovation\Hydraulique\components\230725_PL_V4.41.xlsx')

# create dictionary with dataframes
df_dict = {}
for sheet_name in excel_file.sheet_names:
    df_dict[sheet_name] = excel_file.parse(sheet_name)
    df_dict[sheet_name].drop(columns='Unnamed: 0',inplace=True)

In [ ]:
for i in range(4):
    key = list(df_dict.keys())[i]
    list_Vdot = np.array(df_dict[key]['Vdot_per_panel'])
    list_Dv = list_Vdot*(i+1)/3600000

    path = r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs"
    file_name = f"\V4.41_x{i+1}.xlsx"
    hx, par, cond = fe.initialize(path, file_name)
    list_PL, list_tabl = modf.PL_fsolve_range(par, cond, list_Dv[1::39], 20)

    plt.plot(list_Vdot[1::39], list_PL/1000, label='Simulation')
    plt.plot(list_Vdot, np.array(df_dict[key]['PL']), label='Experiments')
    plt.xlabel('Vdot par panneau [L/h]')
    plt.ylabel('PL [kPa]')
    plt.legend()
    plt.show()



In [ ]:
list(df_dict.keys())

In [ ]:
for k in df_dict.keys():
    print(df_dict[k]['PL'])